Mapping particle tracks from Ocean Parcels unto the Salish Sea Atlantis Boxes. 
Original code written by Bec Gordon & Javier Porobic, CSIRO.
Link to the [SSAM Ocean Parcels Repo](https://bitbucket.csiro.au/users/por07g/repos/ssam_oceanparcels/browse)

In [65]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.path as mplPath
import shapefile
from netCDF4 import Dataset
import dateutil.parser
from shapely.geometry import Polygon, Point

In [66]:
shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/SalishSea_July172019_2/SalishSea_July172019.shp"
data_df = gpd.read_file(shapefile_name)

In [67]:
# Oil type properties & spill location selection

bitumen = {
    "Weight": 1.0219,
    "Naphthalene": 0.024,
    "Phenanthrene": 0.017,
    "Pyrene": 0.010,
    "Benzo": 0.003,
}

BunkerC = {
    "Weight": 1.0178,
    "Naphthalene": 0.680,
    "Phenanthrene": 0.796,
    "Pyrene": 0.266,
    "Benzo": 0.056,
}

Diesel = {
    "Weight": 0.8416,
    "Naphthalene": 3.664,
    "Phenanthrene": 1.000,
    "Pyrene": 0.000,
    "Benzo": 0.000,
}

Crude = {
    "Weight": 0.9336,
    "Naphthalene": 0.654,
    "Phenanthrene": 0.327,
    "Pyrene": 0.013,
    "Benzo": 0.002,
}

fuel_type = {
    "bitumen" : bitumen,
    "BunkerC" : BunkerC,
    "Diesel" : Diesel,
    "Crude" : Crude,
}

file_id = int(input( ))
scenario = {1 : "5b_Turn_Point_Diluted_bitumen",
            2 : "6a_VancouverHarbour_BunkerC",
            3 : "7a_JohnsonStrait_BunkerC",
            4 : "4a_ActivePass_Diesel",}

print("\nScenario running  :", scenario[file_id], sep = " ")


Scenario running  : 5b_Turn_Point_Diluted_bitumen


In [68]:
# Spill release times
release_start = '2018-01-01'  ## winter starts in December, Summer in Jul - Aug
release_end = '2018-01-02'
release_YYYYMM = '2018-01'
spill_volume = 2e6
release_start_time = np.datetime64(release_start)
#inputFileName = scenario[file_id] + str(release_start_time) + '_oil_disperse.nc'
inputFileName = '5b_Turn_Point_Diluted_bitumen2018-01-01_nodecay.nc'


In [69]:
numLayers = 7;
numSites = data_df.shape[0]
numTargetSites = numSites

#outputDT = 60*60
outputDT = 43200

stepsPerDay = int(86400.0/ outputDT);
#numStepsPerDT = stepsPerDay;
numStepsPerDT = int(outputDT/3600.0)

debug = False

In [70]:
pfile = xr.open_dataset(str(inputFileName), decode_cf=True)

In [71]:
lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)
z = np.ma.filled(pfile.variables['z'], np.nan)
probs = np.ma.filled(pfile.variables['decay_value'], np.nan)

In [72]:
numParticles = lon.shape[0]

trackDates = [];

for i in range(0,numParticles):
    #print(time[i][0])
    #trackDates.append( dateutil.parser.parse(time[i][0]))
    trackDates.append(time[i][0])

RDiff = max(trackDates) - min(trackDates);

minDate = np.datetime64(release_start+"T00:30:00");
ts = pd.to_datetime(str(minDate));
d = ts.strftime('%Y-%m-%d %H:%M:%S');
print(d)

2018-01-01 00:30:00


In [73]:
RDiff = max(trackDates) - min(trackDates)

minDate = np.datetime64(release_start+"T00:30:00");
ts = pd.to_datetime(str(minDate))
d = ts.strftime('%Y-%m-%d %H:%M:%S')
print(d)

2018-01-01 00:30:00


In [74]:
numReleaseDays = 1;
numReleaseSteps = numReleaseDays * stepsPerDay;
trackLength = len(lon[0]);

print('trackLength = ' + str(trackLength))
print('numStepsPerDT = ' + str(numStepsPerDT))

numSteps = int(trackLength / numStepsPerDT)


trackLength = 145
numStepsPerDT = 12


In [79]:
# Create the netcdf output file

#netcdfFileName = "Atlantis_" + scenario[file_id] + str(release_start_time) + str(outputDT) + ".nc"
netcdfFileName = "Atlantis_TurnPoint_20180101_43200_1depth.nc"
try:
    os.remove(netcdfFileName)
except:
    pass

ncfile = Dataset(netcdfFileName, "w", format="NETCDF4", clobber=True)

# Dimensions
time = ncfile.createDimension("t", None)
b = ncfile.createDimension("b", numTargetSites)
z = ncfile.createDimension("z", numLayers)

In [80]:
# Variables
times = ncfile.createVariable("t",np.float64, ("t",))
oil = ncfile.createVariable("oil",np.float64,("t", "b", "z"))
Naphthalene = ncfile.createVariable("Naphthalene",np.float64,("t", "b", "z"))
Phenanthrene = ncfile.createVariable("Phenanthrene",np.float64,("t", "b", "z"))
Pyrene = ncfile.createVariable("Pyrene",np.float64,("t", "b", "z"))
Benzo = ncfile.createVariable("Benzo",np.float64,("t", "b", "z"))

# Attributes
Naphthalene.units = "mgPAH/m^3"
Naphthalene.long_name = "Naphthalene"
#Naphthalene._FillValue = 0.0
Naphthalene.missing_value = 0.0
Naphthalene.valid_min = 0.0
Naphthalene.valid_max = 100000000.0

Phenanthrene.units = "mgPAH/m^3"
Phenanthrene.long_name = "Phenanthrene"
#Phenanthrene._FillValue = 0.0
Phenanthrene.missing_value = 0.0
Phenanthrene.valid_min = 0.0
Phenanthrene.valid_max = 100000000.0

Pyrene.units = "mgPAH/m^3"
Pyrene.long_name = "Pyrene"
#Pyrene._FillValue = 0.0
Pyrene.missing_value = 0.0
Pyrene.valid_min = 0.0
Pyrene.valid_max = 100000000.0

Benzo.units = "mgPAH/m^3"
Benzo.long_name = "Benzo"
#Benzo._FillValue = 0.0
Benzo.missing_value = 0.0
Benzo.valid_min = 0.0
Benzo.valid_max = 100000000.0

oil.units = "gOil/m^3"
oil.long_name = "Oil"

times.units = "seconds since 1950-01-01 00:00:00 +10"
times.dt = outputDT
#t.dt = 43200
#t.dt = 3600
times.long_name = "time"

# Populate variables with data
timeData = np.arange(0,(numSteps + numReleaseSteps)*outputDT,outputDT)
times[:] = timeData;
#t[:] = timeData[-1];

boxDispersal = np.zeros((numSteps + numReleaseSteps, numTargetSites));

print(boxDispersal)

In [82]:
for partIndex in range(0, numParticles):

    trackDateDiff = trackDates[partIndex] - minDate;
    trackDateDiff = trackDateDiff/ np.timedelta64(1, 's')

    timeOffset = int(abs((trackDateDiff /outputDT)));

    for stepIndex in range(0, numSteps):
        timeValue = stepIndex + timeOffset

        partLon = lon[partIndex][stepIndex * numStepsPerDT];
        partLat = lat[partIndex][stepIndex * numStepsPerDT];
        partProb = probs[partIndex][stepIndex * numStepsPerDT];

        matchFound = 0;

        for targetIndex in range(0, numTargetSites):

            path = data_df.iloc[targetIndex].geometry
            checks = path.contains(Point(partLon, partLat));

            if checks:
                boxDispersal[timeValue][targetIndex] = boxDispersal[timeValue][targetIndex] + partProb;

                # uncomment line below to ignore particle decay during debugging.
                #boxDispersal[timeValue][targetIndex] = boxDispersal[timeValue][targetIndex] + 1.0

                matchFound = 1
                if debug:
                    print('At time ' + str(timeValue) + ' Particle (' + str(partIndex) + ') in box ' + str(data_df.iloc[targetIndex].BOX_ID))


                break;

        if matchFound == 0:
            if debug:
                print('No match for particle')
                print(partLon, partLat)


        #break

oil[:, :, 5] = boxDispersal * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Weight"] * spill_volume;
Naphthalene[:, :, :] = oil[:, :] * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Naphthalene"];
Phenanthrene[:, :, :] = oil[:, :] * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Phenanthrene"];
Pyrene[:, :, :] = oil[:, :] * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Pyrene"];
Benzo[:, :, :] = oil[:, :] * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Benzo"];

ncfile.close()